In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here are several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/map-charting-student-math-misunderstandings/sample_submission.csv
/kaggle/input/map-charting-student-math-misunderstandings/train.csv
/kaggle/input/map-charting-student-math-misunderstandings/test.csv


In [2]:
# !pip install -q \
#   transformers==4.38.2 \
#   datasets==2.18.0 \
#   accelerate==0.27.2 \
#   peft==0.9.0 \
#   evaluate==0.4.1


In [3]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed,
)


2026-01-29 12:47:38.713681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769690859.015640      95 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769690859.108112      95 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769690859.817663      95 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769690859.817717      95 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769690859.817719      95 computation_placer.cc:177] computation placer alr

In [4]:
DATA_DIR = "/kaggle/input/map-charting-student-math-misunderstandings"

train_path = os.path.join(DATA_DIR, "train.csv")
test_path  = os.path.join(DATA_DIR, "test.csv")
sub_path   = os.path.join(DATA_DIR, "sample_submission.csv")

train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)
sub_df   = pd.read_csv(sub_path)

print(train_df.shape, test_df.shape)
train_df.head()


(36696, 7) (3, 5)


,row_id,QuestionId,QuestionText,MC_Answer,StudentExplanation,Category,Misconception
0,0,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),0ne third is equal to tree nineth,True_Correct,NaN
1,1,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 / 3 because 6 over 9 is 2 thirds and 1 third...,True_Correct,NaN
2,2,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),"1 3rd is half of 3 6th, so it is simplee to un...",True_Neither,NaN
3,3,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 goes into everything and 3 goes into nine,True_Neither,NaN
4,4,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 out of every 3 isn't coloured,True_Correct,NaN


In [5]:
train_df["Category"].unique()

array(['True_Correct', 'True_Neither', 'True_Misconception',
       'False_Neither', 'False_Misconception', 'False_Correct'],
      dtype=object)

In [6]:
train_df["Misconception"].unique()

array([nan, 'Incomplete', 'WNB', 'SwapDividend', 'Mult', 'FlipChange',
       'Irrelevant', 'Wrong_Fraction', 'Additive', 'Not_variable',
       'Adding_terms', 'Inverse_operation', 'Inversion', 'Duplication',
       'Wrong_Operation', 'Whole_numbers_larger', 'Longer_is_bigger',
       'Ignores_zeroes', 'Shorter_is_bigger', 'Wrong_fraction',
       'Adding_across', 'Denominator-only_change',
       'Incorrect_equivalent_fraction_addition', 'Division',
       'Subtraction', 'Unknowable', 'Definition', 'Interior', 'Positive',
       'Tacking', 'Wrong_term', 'Firstterm', 'Base_rate',
       'Multiplying_by_4', 'Certainty', 'Scale'], dtype=object)

In [7]:
train_df["Misconception"] = train_df["Misconception"].fillna("NA")

train_df["label_text"] = (
    train_df["Category"].astype(str)
    + ":" +
    train_df["Misconception"].astype(str)
)

In [8]:
train_df["Misconception"].unique()

array(['NA', 'Incomplete', 'WNB', 'SwapDividend', 'Mult', 'FlipChange',
       'Irrelevant', 'Wrong_Fraction', 'Additive', 'Not_variable',
       'Adding_terms', 'Inverse_operation', 'Inversion', 'Duplication',
       'Wrong_Operation', 'Whole_numbers_larger', 'Longer_is_bigger',
       'Ignores_zeroes', 'Shorter_is_bigger', 'Wrong_fraction',
       'Adding_across', 'Denominator-only_change',
       'Incorrect_equivalent_fraction_addition', 'Division',
       'Subtraction', 'Unknowable', 'Definition', 'Interior', 'Positive',
       'Tacking', 'Wrong_term', 'Firstterm', 'Base_rate',
       'Multiplying_by_4', 'Certainty', 'Scale'], dtype=object)

In [9]:
labels = sorted(train_df["label_text"].unique())

In [10]:
labels

['False_Correct:NA',
 'False_Misconception:Adding_across',
 'False_Misconception:Adding_terms',
 'False_Misconception:Additive',
 'False_Misconception:Base_rate',
 'False_Misconception:Certainty',
 'False_Misconception:Definition',
 'False_Misconception:Denominator-only_change',
 'False_Misconception:Division',
 'False_Misconception:Duplication',
 'False_Misconception:Firstterm',
 'False_Misconception:FlipChange',
 'False_Misconception:Ignores_zeroes',
 'False_Misconception:Incomplete',
 'False_Misconception:Incorrect_equivalent_fraction_addition',
 'False_Misconception:Interior',
 'False_Misconception:Inverse_operation',
 'False_Misconception:Inversion',
 'False_Misconception:Irrelevant',
 'False_Misconception:Longer_is_bigger',
 'False_Misconception:Mult',
 'False_Misconception:Multiplying_by_4',
 'False_Misconception:Not_variable',
 'False_Misconception:Positive',
 'False_Misconception:Scale',
 'False_Misconception:Shorter_is_bigger',
 'False_Misconception:Subtraction',
 'False_Misc

In [11]:
label2id = {l:i for i,l in enumerate(labels)}

In [12]:
label2id

{'False_Correct:NA': 0,
 'False_Misconception:Adding_across': 1,
 'False_Misconception:Adding_terms': 2,
 'False_Misconception:Additive': 3,
 'False_Misconception:Base_rate': 4,
 'False_Misconception:Certainty': 5,
 'False_Misconception:Definition': 6,
 'False_Misconception:Denominator-only_change': 7,
 'False_Misconception:Division': 8,
 'False_Misconception:Duplication': 9,
 'False_Misconception:Firstterm': 10,
 'False_Misconception:FlipChange': 11,
 'False_Misconception:Ignores_zeroes': 12,
 'False_Misconception:Incomplete': 13,
 'False_Misconception:Incorrect_equivalent_fraction_addition': 14,
 'False_Misconception:Interior': 15,
 'False_Misconception:Inverse_operation': 16,
 'False_Misconception:Inversion': 17,
 'False_Misconception:Irrelevant': 18,
 'False_Misconception:Longer_is_bigger': 19,
 'False_Misconception:Mult': 20,
 'False_Misconception:Multiplying_by_4': 21,
 'False_Misconception:Not_variable': 22,
 'False_Misconception:Positive': 23,
 'False_Misconception:Scale': 24,


In [13]:
id2label = {i:l for l,i in label2id.items()}

In [14]:
id2label

{0: 'False_Correct:NA',
 1: 'False_Misconception:Adding_across',
 2: 'False_Misconception:Adding_terms',
 3: 'False_Misconception:Additive',
 4: 'False_Misconception:Base_rate',
 5: 'False_Misconception:Certainty',
 6: 'False_Misconception:Definition',
 7: 'False_Misconception:Denominator-only_change',
 8: 'False_Misconception:Division',
 9: 'False_Misconception:Duplication',
 10: 'False_Misconception:Firstterm',
 11: 'False_Misconception:FlipChange',
 12: 'False_Misconception:Ignores_zeroes',
 13: 'False_Misconception:Incomplete',
 14: 'False_Misconception:Incorrect_equivalent_fraction_addition',
 15: 'False_Misconception:Interior',
 16: 'False_Misconception:Inverse_operation',
 17: 'False_Misconception:Inversion',
 18: 'False_Misconception:Irrelevant',
 19: 'False_Misconception:Longer_is_bigger',
 20: 'False_Misconception:Mult',
 21: 'False_Misconception:Multiplying_by_4',
 22: 'False_Misconception:Not_variable',
 23: 'False_Misconception:Positive',
 24: 'False_Misconception:Scale',


In [15]:
train_df["label"] = train_df["label_text"].map(label2id)

In [16]:
train_df["label"].unique()

array([37, 64, 47, 60, 36, 30, 13,  0, 27, 20, 11, 58, 52, 46, 18, 32,  3,
       50, 39, 54,  2, 22, 16, 49, 44,  9, 17, 33, 31, 19, 12, 25, 61, 56,
       51, 62, 34, 38, 42, 48,  7, 14,  1, 43, 57, 26,  8, 29,  6, 15, 41,
       23, 28, 59, 55, 35, 10, 63, 45,  4, 40, 53, 21,  5, 24])

In [17]:
print("Num classes:", len(labels))

Num classes: 65


In [18]:
num_labels = len(labels)

In [19]:
train_df[["Category","Misconception","label_text","label"]].head()

,Category,Misconception,label_text,label
0,True_Correct,NA,True_Correct:NA,37
1,True_Correct,NA,True_Correct:NA,37
2,True_Neither,NA,True_Neither:NA,64
3,True_Neither,NA,True_Neither:NA,64
4,True_Correct,NA,True_Correct:NA,37


In [20]:
def clean_text(s):
    if pd.isna(s):
        return ""
    s = str(s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [21]:
def make_input(row):
    q = clean_text(row["QuestionText"])
    a = clean_text(row["MC_Answer"])
    e = clean_text(row["StudentExplanation"])
    return f"[QUESTION] {q} [ANSWER_SELECTED] {a} [EXPLANATION] {e}"

In [22]:
train_df["text"] = train_df.apply(make_input, axis=1)
test_df["text"]  = test_df.apply(make_input, axis=1)

train_df[["text","label_text"]].head()

,text,label_text
0,[QUESTION] What fraction of the shape is not s...,True_Correct:NA
1,[QUESTION] What fraction of the shape is not s...,True_Correct:NA
2,[QUESTION] What fraction of the shape is not s...,True_Neither:NA
3,[QUESTION] What fraction of the shape is not s...,True_Neither:NA
4,[QUESTION] What fraction of the shape is not s...,True_Correct:NA


In [23]:
train_df["text"][2]

'[QUESTION] What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle split into 9 equal smaller triangles. 6 of them are shaded.] [ANSWER_SELECTED] \\( \\frac{1}{3} \\) [EXPLANATION] 1 3rd is half of 3 6th, so it is simplee to understand.'

In [24]:
def mapk_single(y_true, y_pred_topk):
    for i, p in enumerate(y_pred_topk):
        if p == y_true:
            return 1.0 / (i + 1)
    return 0.0

In [25]:
def map3_metric(eval_pred):
    logits, y_true = eval_pred
    top3 = np.argsort(-logits, axis=1)[:, :3]
    scores = [mapk_single(t, p.tolist()) for t, p in zip(y_true, top3)]
    return {"map@3": float(np.mean(scores))}

In [26]:
set_seed(42)

N_FOLDS = 5
FOLD = 0  

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
splits = list(skf.split(train_df, train_df["label"]))

train_idx, valid_idx = splits[FOLD]

tr_df = train_df.iloc[train_idx].reset_index(drop=True)
va_df = train_df.iloc[valid_idx].reset_index(drop=True)

print("train fold:", tr_df.shape, "valid fold:", va_df.shape)

train fold: (29356, 10) valid fold: (7340, 10)


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [27]:
MODEL_NAME = "microsoft/deberta-v3-small"   # stable + fast
MAX_LEN = 384

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

from datasets import Dataset

train_ds = Dataset.from_pandas(tr_df[["text", "label"]], preserve_index=False)
valid_ds = Dataset.from_pandas(va_df[["text", "label"]], preserve_index=False)
test_ds  = Dataset.from_pandas(test_df[["row_id", "text"]], preserve_index=False)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [28]:
def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN,
    )

In [29]:
train_ds = train_ds.map(tokenize_batch, batched=True, remove_columns=["text"])
valid_ds = valid_ds.map(tokenize_batch, batched=True, remove_columns=["text"])
test_ds  = test_ds.map(tokenize_batch, batched=True, remove_columns=["text"])

Map:   0%|          | 0/29356 [00:00<?, ? examples/s]

Map:   0%|          | 0/7340 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [30]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [31]:
num_labels = len(labels)

In [32]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
)

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
from transformers import TrainingArguments

args = TrainingArguments(
    
    output_dir="/kaggle/working/outputs",

    # optimization
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,

    # batch sizes
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,

    
    max_steps=1200,
    num_train_epochs=1,

    
    eval_strategy="steps",
    eval_steps=600,

    
    save_strategy="steps",
    save_steps=600,
    save_total_limit=1,

    
    load_best_model_at_end=True,
    metric_for_best_model="map@3",
    greater_is_better=True,

    
    fp16=True,
    dataloader_num_workers=2,
    logging_steps=200,

    report_to="none",
)

In [34]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=map3_metric,
)

trainer.train()

/tmp/ipykernel_95/3744977470.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
hu

model.safetensors:   0%|          | 0.00/286M [00:00<?, ?B/s]

Step,Training Loss,Validation Loss,Map@3
600,1.890500,1.768662,0.620413
1200,1.539800,1.405621,0.714578


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avo

TrainOutput(global_step=1200, training_loss=1.9610706329345704, metrics={'train_runtime': 4656.235, 'train_samples_per_second': 2.062, 'train_steps_per_second': 0.258, 'total_flos': 260593943132016.0, 'train_loss': 1.9610706329345704, 'epoch': 0.32697547683923706})

In [37]:
pred = trainer.predict(test_ds)
logits = pred.predictions

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [42]:
pred

PredictionOutput(predictions=array([[ 5.4005790e-01, -1.3449496e-01, -7.9932797e-01,  6.5722957e-02,
        -1.2348342e+00, -1.9363179e+00, -1.6800187e+00,  3.7401926e-02,
        -2.2090540e+00,  1.0212629e+00, -8.1256175e-01, -5.8315134e-01,
        -1.7266865e+00,  6.1212510e-01, -2.3393507e+00, -1.6498415e+00,
        -1.4366584e+00,  6.4478838e-01,  1.6424464e-01, -2.5124316e+00,
         3.3209872e-01, -3.5257804e-01, -2.1379030e-01,  1.2581204e-01,
        -1.2938584e-01, -1.8667004e+00,  5.6477970e-01, -1.7834541e-01,
        -2.8845331e-01, -6.0550565e-01,  4.5775987e-02, -8.1827962e-01,
         7.3905373e-01, -2.1711504e+00,  1.0208565e+00,  6.2679565e-01,
         3.1404078e+00,  6.5832577e+00, -2.2418482e+00, -1.8262284e+00,
        -1.6158175e+00, -1.6553850e+00, -2.0607259e+00, -1.7870414e+00,
        -1.6840358e+00, -1.7102848e+00, -1.9865162e+00, -1.9771843e+00,
        -2.2006726e+00, -2.5554790e+00, -8.8391507e-01, -1.9547533e+00,
        -1.4466455e+00, -2.2834628e

In [43]:
logits

array([[ 5.4005790e-01, -1.3449496e-01, -7.9932797e-01,  6.5722957e-02,
        -1.2348342e+00, -1.9363179e+00, -1.6800187e+00,  3.7401926e-02,
        -2.2090540e+00,  1.0212629e+00, -8.1256175e-01, -5.8315134e-01,
        -1.7266865e+00,  6.1212510e-01, -2.3393507e+00, -1.6498415e+00,
        -1.4366584e+00,  6.4478838e-01,  1.6424464e-01, -2.5124316e+00,
         3.3209872e-01, -3.5257804e-01, -2.1379030e-01,  1.2581204e-01,
        -1.2938584e-01, -1.8667004e+00,  5.6477970e-01, -1.7834541e-01,
        -2.8845331e-01, -6.0550565e-01,  4.5775987e-02, -8.1827962e-01,
         7.3905373e-01, -2.1711504e+00,  1.0208565e+00,  6.2679565e-01,
         3.1404078e+00,  6.5832577e+00, -2.2418482e+00, -1.8262284e+00,
        -1.6158175e+00, -1.6553850e+00, -2.0607259e+00, -1.7870414e+00,
        -1.6840358e+00, -1.7102848e+00, -1.9865162e+00, -1.9771843e+00,
        -2.2006726e+00, -2.5554790e+00, -8.8391507e-01, -1.9547533e+00,
        -1.4466455e+00, -2.2834628e+00, -2.1363509e+00, -2.07701

In [38]:
top3 = np.argsort(-logits, axis=1)[:, :3]

top3_labels = [
    " ".join([id2label[int(i)] for i in row])
    for row in top3
]

In [41]:
submission = pd.DataFrame({
    "row_id": test_df["row_id"],
    "Category:Misconception": top3_labels
})

submission.to_csv("submission.csv", index=False)
print("submission.csv saved")


submission.csv saved


In [44]:
import numpy as np
from scipy.special import softmax

probs = softmax(logits, axis=1)

**For Analysis**

In [45]:
top3_idx = np.argsort(-probs, axis=1)[:, :3]
top3_scores = np.take_along_axis(probs, top3_idx, axis=1)

In [46]:
top3_preds = [
    [id2label[int(i)] for i in row]
    for row in top3_idx
]

In [47]:
analysis_df = test_df.copy()

analysis_df["pred_1"] = [p[0] for p in top3_preds]
analysis_df["pred_2"] = [p[1] for p in top3_preds]
analysis_df["pred_3"] = [p[2] for p in top3_preds]

analysis_df["score_1"] = [s[0] for s in top3_scores]
analysis_df["score_2"] = [s[1] for s in top3_scores]
analysis_df["score_3"] = [s[2] for s in top3_scores]

In [48]:
analysis_df.head()

,row_id,QuestionId,QuestionText,MC_Answer,StudentExplanation,text,pred_1,pred_2,pred_3,score_1,score_2,score_3
0,36696,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),"I think that 1/3 is the answer, as it's the si...",[QUESTION] What fraction of the shape is not s...,True_Correct:NA,True_Neither:NA,False_Neither:NA,0.839784,0.089603,0.026851
1,36697,31772,What fraction of the shape is not shaded? Give...,\( \frac{3}{6} \),i think this answer is because 3 triangles are...,[QUESTION] What fraction of the shape is not s...,False_Misconception:Incomplete,False_Neither:NA,True_Neither:NA,0.648683,0.148564,0.032942
2,36698,32835,Which number is the greatest?,\( 6.2 \),because the 2 makes it higher than the others.,[QUESTION] Which number is the greatest? [ANSW...,True_Neither:NA,True_Correct:NA,False_Neither:NA,0.439295,0.353016,0.107765


In [52]:
analysis_df["text"][2]

'[QUESTION] Which number is the greatest? [ANSWER_SELECTED] \\( 6.2 \\) [EXPLANATION] because the 2 makes it higher than the others.'